# Rewrite GAMS codes in Python: A brief comparison

  + Author: Sheng Dai (sheng.dai@aalto.fi)
  + Date  : April 15, 2020

[PYOMO](http://www.pyomo.org/) provides a good coding environment that can help us smoothly transfer from GAMS to Python. Thus, we prepare a short tutorial to help GAMSers to understand how to rewrite the StoNED models, even other complicated models on Python.

### Import the pyomo module

In [1]:
from pyomo.environ import *

# Create a concrete Model
model = ConcreteModel(name = "CNLS")

### SETS

In [2]:
#sets 
#     i        "DMU's"  /1*10/
#     j        'outputs' /Energy, Length, Customers/

In [3]:
model.i = Set(initialize=['i1', 'i2', 'i3', 'i4', 'i5', 'i6','i7', 'i8', 'i9','i10'], doc='DMUS', ordered=True)
model.j = Set(initialize=['Energy', 'Length', 'Customers'], doc='outputs')

### ALIAS

In [4]:
# alias(i,h);

In [5]:
model.h = SetOf(model.i)

### TABLE

In [6]:
#Table data(i,j)
#$Include energy.txt;

In [7]:
dtab = {
    ('i1',  'Energy')   : 75,		
    ('i1',  'Length')   : 878,
    ('i1',  'Customers'): 4933,
    ('i2',  'Energy')   : 62,
    ('i2',  'Length')   : 964,
    ('i2',  'Customers'): 6149,
    ('i3',  'Energy')   : 78,
    ('i3',  'Length')   : 676,
    ('i3',  'Customers'): 6098,
    ('i4',  'Energy')   : 683,
    ('i4',  'Length')   : 12522,
    ('i4',  'Customers'): 55226,
    ('i5',  'Energy')   : 27,
    ('i5',  'Length')   : 697,
    ('i5',  'Customers'): 1670,
    ('i6',  'Energy')   : 295,
    ('i6',  'Length')   : 953,
    ('i6',  'Customers'): 22949,
    ('i7',  'Energy')   : 44,
    ('i7',  'Length')   : 917,
    ('i7',  'Customers'): 3599,
    ('i8',  'Energy')   : 171,
    ('i8',  'Length')   : 1580,
    ('i8',  'Customers'): 11081,
    ('i9',  'Energy')   : 98,
    ('i9',  'Length')   : 116,
    ('i9',  'Customers'): 377,
    ('i10',  'Energy')   : 203,
    ('i10',  'Length')   : 740,
    ('i10',  'Customers'): 10134,
    }
model.d = Param(model.i, model.j, initialize=dtab, doc='output data')

### PARAMETERS

In [8]:
#PARAMETERS
#c(i)         "Total cost of firm i"
#y(i,j)       "Output j of firm i";

In [9]:
model.c = Param(model.i, initialize={'i1': 1612,
                                     'i2': 1659,
                                     'i3': 1708,
                                     'i4': 18918,
                                     'i5': 1167,
                                     'i6': 3395,
                                     'i7': 1333,
                                     'i8': 3518,
                                     'i9': 1415,
                                     'i10':2469,
                                     }, 
                doc='Cost data')

def y_init(model, i, j):
  return  model.d[i, j]
model.y = Param(model.i, model.j, initialize=y_init, doc='output data')

### VARIABLES

In [10]:
#VARIABLES
#E(i)            "Composite error term (v + u)"
#SSE             "Sum of squares of residuals";

#POSITIVE VARIABLES
#b(i,j)    "Beta-coefficients (positivity ensures monotonicity)"
#Chat(i)  ;

In [11]:
model.b = Var(model.i, model.j, bounds=(0.0,None), doc='beta-coeff')
model.e = Var(model.i, doc='res')
model.f = Var(model.i, bounds=(0.0,None), doc='frontier')

### EQUATIONS

In [12]:
#Equations
#QSSE                  objective function = sum of squares of residuals
#QREGRESSION(i)        log-transformed regression equation
#Qlog(i)               supporting hyperplanes of the nonparametric cost function
#QCONC(i,h)            concavity constraint (Afriat inequalities);

#QSSE..                SSE=e=sum(i,E(i)*E(i)) ;
#QREGRESSION(i)..      log(C(i)) =e= log(Chat(i) + 1) + E(i);
#Qlog(i)..             Chat(i) =e= sum(j, b(i,j)*Y(i,j)) - 1;
#QCONC(i,h)..          sum(j, b(i,j)*Y(i,j)) =g= sum(j, b(h,j)*Y(i,j));

In [13]:
def objective_rule(model):
  return sum(model.e[i]*model.e[i] for i in model.i)
model.objective = Objective(rule=objective_rule, sense=minimize, doc='objective function')

def qreg_rule(model, i):
    return log(model.c[i]) == log(model.f[i] + 1) + model.e[i]
model.qreg = Constraint(model.i, rule=qreg_rule, doc='log-transformed regression')

def qlog_rule(model, i):
    return model.f[i] == sum(model.b[i, j]*model.y[i, j] for j in model.j) - 1
model.qlog = Constraint(model.i, rule=qlog_rule, doc='cost function')

def qconvex_rule(model, i, h):
    return sum(model.b[i,j]*model.y[i,j] for j in model.j) >= sum(model.b[h,j]*model.y[i,j] for j in model.j)
model.qconvex = Constraint(model.i, model.h, rule=qconcav_rule, doc='convexity constraint')

### EXECUTION

In [14]:
# Execute the model
#MODEL StoNED /all/;
#SOLVE StoNED using NLP Minimizing SSE;

In [15]:
from pyomo.opt import SolverFactory
import pyomo.environ
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='minos')

### DISPLAY

In [16]:
#display E.l, b.l;

In [17]:
model.e.display()
model.b.display()  
model.f.display() 

e : res
    Size=10, Index=i
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
     i1 :  None : -0.032096679396871504 :  None : False : False :  Reals
    i10 :  None :  -0.06413156735462289 :  None : False : False :  Reals
     i2 :  None : -0.009706295145954136 :  None : False : False :  Reals
     i3 :  None :   0.14091387439378697 :  None : False : False :  Reals
     i4 :  None :  -0.07357647574238865 :  None : False : False :  Reals
     i5 :  None :   0.06750602546432768 :  None : False : False :  Reals
     i6 :  None :  -0.11941039400503946 :  None : False : False :  Reals
     i7 :  None :  -0.07382034813946062 :  None : False : False :  Reals
     i8 :  None :   0.05689878964359083 :  None : False : False :  Reals
     i9 :  None :   0.10742325374875122 :  None : False : False :  Reals
b : beta-coeff
    Size=30, Index=b_index
    Key                  : Lower : Value                : Upper : Fixed : Stale : Domain
     ('i1', 'Customers') :   0.0 :  0

In [18]:
import pandas as pd    
E = {(i, v.name): value(v) for (i), v in model.e.items()}
df_E = pd.DataFrame.from_dict(E, orient="index", columns=["residuals"])
print(df_E)

B = {(i, v.name): value(v) for (i, j), v in model.b.items()}
df_B = pd.DataFrame.from_dict(B, orient="index", columns=["beta-coeff"])
print(df_B)

coeff = pd.concat([df_E, df_B], axis=1) 

               residuals
(i1, e[i1])    -0.032097
(i2, e[i2])    -0.009706
(i3, e[i3])     0.140914
(i4, e[i4])    -0.073576
(i5, e[i5])     0.067506
(i6, e[i6])    -0.119410
(i7, e[i7])    -0.073820
(i8, e[i8])     0.056899
(i9, e[i9])     0.107423
(i10, e[i10])  -0.064132
                         beta-coeff
(i1, b[i1,Energy])         8.088506
(i1, b[i1,Length])         1.111964
(i1, b[i1,Customers])      0.016549
(i2, b[i2,Energy])         0.000000
(i2, b[i2,Length])         0.917438
(i2, b[i2,Customers])      0.128602
(i3, b[i3,Energy])         8.088506
(i3, b[i3,Length])         1.111964
(i3, b[i3,Customers])      0.016549
(i4, b[i4,Energy])         8.088506
(i4, b[i4,Length])         1.111964
(i4, b[i4,Customers])      0.016549
(i5, b[i5,Energy])         0.000000
(i5, b[i5,Length])         1.565022
(i5, b[i5,Customers])      0.000000
(i6, b[i6,Energy])         0.000000
(i6, b[i6,Length])         0.917437
(i6, b[i6,Customers])      0.128602
(i7, b[i7,Energy])         0.000000
(i7, 